In [91]:
import requests
from bs4 import BeautifulSoup
from pypdf import PdfReader
import pandas as pd
from datetime import date, timedelta

from selenium.common import NoSuchElementException

from filename_utils import sanitize_filename

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [99]:
from selenium.common import StaleElementReferenceException
import re


def visit_page(url, day, hidden=True):
    options = Options()
    if hidden:
        options.add_argument("--headless=new")
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)

    driver.get(url)

    # Select dropdown
    select = Select(wait.until(EC.element_to_be_clickable((By.ID, "id3"))))
    select.select_by_value("0")

    # Wait until Wicket loads the expanded form
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#id1 form")))
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#id1 form")))

    # Set Von
    von = wait.until(EC.element_to_be_clickable((By.ID, "id7")))
    von.clear()
    von.send_keys(day)
    von.send_keys(Keys.TAB)  # Important: trigger onchange event

    # Set Bis
    bis = wait.until(EC.element_to_be_clickable((By.ID, "id8")))
    bis.clear()
    bis.send_keys(day)
    bis.send_keys(Keys.TAB)

    # Re-find the new submit button created by Wicket
    search_button = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#id1 form button[type=submit]"))
    )

    # Wait until button is present AND clickable
    search_button = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#id1 form button[type=submit]"))
    )

    # Scroll button into view
    driver.execute_script("arguments[0].scrollIntoView(true);", search_button)

    # Click using JS (forces Wicket to receive the click)
    driver.execute_script("arguments[0].click();", search_button)

    # Wait for results container to reload
    try:
        wait.until(lambda d: d.find_element(By.ID, "id2").get_attribute("hidden") is None)
    except NoSuchElementException and StaleElementReferenceException:
        pass
    # Wicket rewrote the DOM → re-locate the dropdown fresh
    items_per_page_dropdown = wait.until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "select.itemsperpage"))
    )
    if ("Es wurden keine Einträge gefunden!" not in driver.page_source
    and (m := re.search(r"1\s*-\s*10\s+von\s+(\d+)", driver.page_source))
    and int(m.group(1)) > 10):
        # Change to 100 results
        select_items = Select(items_per_page_dropdown)
        select_items.select_by_value("3")  # = “100”

        # Wait for Wicket to reload results again
        wait.until(lambda d: "1 - 10 von" not in d.page_source)

    html = driver.page_source
    driver.quit()
    return html


In [93]:
def get_meta_data(soup_item):
    data = {}

    # get name
    div = soup_item.find("div", class_="flex-grow-1")
    name = div.get_text(separator=" ", strip=True)
    data["name"] = name

    # get metadata
    rows = soup_item.find_all("div", class_="keyvalue-row")
    for row in rows:
        key_div = row.find("div", class_="keyvalue-key")
        value_div = row.find("div", class_="keyvalue-value")
        key = key_div.get_text(strip=True).rstrip(":")
        value = value_div.get_text(" ", strip=True)

        data[key] = value

    # get pdf link
    base = "https://risi.muenchen.de/risi/"
    a = soup_item.find("a", class_= "downloadlink text-nohyphens")
    try:
        pdf_href = a["href"]
    except TypeError:
        pdf_href = "No pdf found"
    data["document_link"] = base + pdf_href


    return name, data

In [94]:
def download_pdf(name, entry, path):
    link = entry['document_link']
    if "No pdf found" not in link:
        response = requests.get(link)
        # Sanitize filename to remove problematic characters for Windows
        sanitized_name = sanitize_filename(name)
        file_path = f'{path}/{sanitized_name}.pdf'

        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)

        reader = PdfReader(file_path)

        for page in reader.pages:
            text = page.extract_text()
            entry['document_content'] = text

In [100]:
# select dates to load items from
start_date = date(2016, 3, 11)
end_date = date(2026, 1, 26)
delta = timedelta(days=1)

#load html from all dates
url = f"https://risi.muenchen.de/risi/erweitertesuche?15&objekt=SITZUNGSVORLAGE;jsessionid=A5933341DAEC20EAFAE1BDD677C67FD8?0"

html_pages = []
error_counter = 0
i = 1
while start_date <= end_date:
    current_day = start_date.strftime("%d.%m.%Y")
    print(f"{i}: Downloading {current_day}...", end = "")

    try:
        html_page = visit_page(url, current_day, hidden=True)
        print("✅")
        start_date += delta
        i += 1
        error_counter = 0

    except TypeError as e:
        error_counter += 1
        print(" ")
        if error_counter > 3:
            print(f"WARNING: {start_date} not fetched successfully.")
            start_date += delta
            i += 1

    if "schränken Sie Ihre Suche ein" in html_page:
        print("WARNING: Not all items fetched successfully.")

    html_pages.append(html_page)


1: Downloading 11.03.2016...✅
2: Downloading 12.03.2016...✅
3: Downloading 13.03.2016...✅
4: Downloading 14.03.2016...✅
5: Downloading 15.03.2016...✅
6: Downloading 16.03.2016...✅
7: Downloading 17.03.2016...✅
8: Downloading 18.03.2016...✅
9: Downloading 19.03.2016...✅
10: Downloading 20.03.2016...✅
11: Downloading 21.03.2016...✅
12: Downloading 22.03.2016...✅
13: Downloading 23.03.2016...✅
14: Downloading 24.03.2016...✅
15: Downloading 25.03.2016...✅
16: Downloading 26.03.2016...✅
17: Downloading 27.03.2016...✅
18: Downloading 28.03.2016...✅
19: Downloading 29.03.2016...✅
20: Downloading 30.03.2016...✅
21: Downloading 31.03.2016...✅
22: Downloading 01.04.2016...✅
23: Downloading 02.04.2016...✅
24: Downloading 03.04.2016...✅
25: Downloading 04.04.2016...✅
26: Downloading 05.04.2016...✅
27: Downloading 06.04.2016...✅
28: Downloading 07.04.2016...✅
29: Downloading 08.04.2016...✅
30: Downloading 09.04.2016...✅
31: Downloading 10.04.2016...✅
32: Downloading 11.04.2016...✅
33: Downloading 1

In [101]:

len(html_pages)

3609

In [102]:
# get all items and fill data dict
data = {}
j=0
for page in html_pages:
    j+=1
    soup = BeautifulSoup(page, "html.parser")
    soup = BeautifulSoup(page, "html.parser")
    soup_items = soup.find_all("li", class_="list-group-item even")
    soup_items += soup.find_all("li", class_="list-group-item odd")

    print("Page",j, "Items:", len(soup_items))

    i = 0
    for item in soup_items:
        i+=1
        name, item_data = get_meta_data(item)
        data[name] = item_data

Page 1 Items: 8
Page 2 Items: 0
Page 3 Items: 0
Page 4 Items: 1
Page 5 Items: 12
Page 6 Items: 10
Page 7 Items: 14
Page 8 Items: 3
Page 9 Items: 0
Page 10 Items: 0
Page 11 Items: 2
Page 12 Items: 2
Page 13 Items: 3
Page 14 Items: 7
Page 15 Items: 0
Page 16 Items: 0
Page 17 Items: 0
Page 18 Items: 0
Page 19 Items: 3
Page 20 Items: 3
Page 21 Items: 4
Page 22 Items: 1
Page 23 Items: 0
Page 24 Items: 0
Page 25 Items: 1
Page 26 Items: 3
Page 27 Items: 3
Page 28 Items: 10
Page 29 Items: 4
Page 30 Items: 0
Page 31 Items: 0
Page 32 Items: 0
Page 33 Items: 5
Page 34 Items: 8
Page 35 Items: 21
Page 36 Items: 3
Page 37 Items: 0
Page 38 Items: 0
Page 39 Items: 0
Page 40 Items: 6
Page 41 Items: 5
Page 42 Items: 7
Page 43 Items: 9
Page 44 Items: 0
Page 45 Items: 0
Page 46 Items: 2
Page 47 Items: 3
Page 48 Items: 6
Page 49 Items: 15
Page 50 Items: 4
Page 51 Items: 0
Page 52 Items: 0
Page 53 Items: 1
Page 54 Items: 4
Page 55 Items: 12
Page 56 Items: 0
Page 57 Items: 9
Page 58 Items: 0
Page 59 Items: 0

In [103]:
print(len(data))

14010


In [104]:
# download all pdfs
i = 0
for name, entry in data.items():
    i +=1
    print(f"Downloading item {i}...", end = "")
    download_pdf(name, entry, "antraege")
    print("✅")

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)


✅

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)


✅

Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)


...

✅

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)


✅

Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)


✅


In [113]:
# fill database
df = pd.DataFrame.from_dict(data, orient="index")
df["Gestellt am"] = pd.to_datetime(df["Gestellt am"], format="%d.%m.%Y")
df = df.sort_values("Gestellt am")

In [114]:
# create csv
df.to_csv("data_2016_now.csv", index=False)

In [115]:
print(len(data))

14010


In [116]:
print(len(df))

14010


In [117]:
df_2006_2009 = pd.read_csv("data_2006_2009.csv")
df_2009_2016 = pd.read_csv("data_2009_2016.csv")
df_2016_2026 = pd.read_csv("data_2016_2026.csv")

# Combine them
combined_df = pd.concat(
    [df_2006_2009, df_2009_2016, df_2016_2026],
    ignore_index=True
)

combined_df.to_csv("data_2006_2026.csv", index=False)